# Agent V3: The Skilled

A monolithic agent with progressive disclosure via skills. Instead of loading all
specialized instructions into the system prompt upfront, the agent discovers skills
at startup (metadata only) and activates them on demand (loading full SKILL.md body).
Extends the Planner with activate_skill: file tools (9) + sandbox tools (1) + todo tools (6) + skill tool (1) = 17 tools.

In [ ]:
from agentic_patterns.core.agents import AgentSpec, OrchestratorAgent
from agentic_patterns.core.workspace import clean_up_session

## Agent Definition

The `OrchestratorAgent` auto-discovers skills from `SKILLS_DIR`, injects the skill catalog
into the system prompt, and adds the `activate_skill` tool. The notebook only provides
the base tools and the system prompt.

In [ ]:
spec = AgentSpec.from_config("skilled")
print(spec)

In [ ]:
clean_up_session()
agent = OrchestratorAgent(spec, verbose=True)

## Turn 1: Write a Script

A normal coding task -- no skills needed yet.

In [ ]:
async with agent:
    prompt_1 = """Write a Python script that reads user input and evaluates it as a math expression.
Save it to /workspace/calc.py and execute it with a test input."""

    result_1 = await agent.run(prompt_1)

    print("\n--- Agent Output ---")
    print(result_1.output)

## Turn 2: Review the Script

Ask the agent to review the script for security issues. The agent should discover the
code-review skill from the catalog and activate it on its own. The `OrchestratorAgent`
carries message history from turn 1 automatically.
Watch for the `[SKILL ACTIVATED: code-review]` marker in the output.

In [ ]:
async with agent:
    prompt_2 = "Now review the script you just wrote for security issues."

    result_2 = await agent.run(prompt_2)

    print("\n--- Agent Output ---")
    print(result_2.output)

## Verify Workspace

In [ ]:
from agentic_patterns.core.workspace import list_workspace_files, read_from_workspace

for path in sorted(list_workspace_files("*")):
    try:
        content = read_from_workspace(path)
    except UnicodeDecodeError:
        content = "<binary file>"
    print(f"--- {path} ---")
    print(content)
    print()